# Notebook contains a simple possible magnetic trap configuration

In [ ]:
%run config.py

In [ ]:
from ford1991 import solve_3d
from mpl_toolkits import mplot3d
from scipy.constants import c, electron_mass as me, elementary_charge as qe, mu_0 as mu0
from qtnm_fields import BathTubField

In [ ]:
# Deflection from beam direction in degrees
theta_deg = -1
theta = np.deg2rad(theta_deg)
# Initial kinetic energy (eV)
T = 18600
# Rel. gamma
gamma_rel = T * qe / (me*c**2) + 1
# (v/c)^2
beta_sq = 1 - 1 / gamma_rel**2
# Initial electron speed
v0 = np.sqrt(beta_sq) * c
# Background B-field (T)
B0 = np.array([0, 0, 1.0])
# Switch off Larmor term
tau = 0.0

## Simple simulation. Uniform B-field - perpendicular to beam

## Initialise electron with direction -1 degree relative to beam

In [ ]:
ic = [0, 0, 0, v0 * np.cos(theta), 0, v0 * np.sin(theta), 0.0]
res = solve_3d(100, b0=B0, v0=v0, mass=me, charge=-qe, tau=tau, ic=ic)

In [ ]:
## Plot results
x = res.y[0]
y = res.y[1]
z = res.y[2]
fig = plt.figure(figsize=[18,12])
ax0 = fig.add_subplot(1,3,1, projection='3d')
ax0.plot(x, y, z)
ax0.view_init(elev=15, azim=-30)
ax1 = fig.add_subplot(1,3,2, projection='3d')
ax1.plot(x, y, z)
ax1.view_init(elev=0, azim=90)
ax2 = fig.add_subplot(1,3,3, projection='3d')
ax2.plot(x, y, z)
ax2.view_init(elev=90, azim=0)

In [ ]:
plt.axhspan(-0.5, 0.5, color='grey', alpha=0.25)
plt.ylim(-2,2.75)

ax = plt.gca()

xp = np.linspace(0.1,1,10)

for b in xp:
    ax.annotate('', xy=(b, -2), xytext=(0, 130), textcoords='offset points', 
                arrowprops=dict(arrowstyle="<|-", color='blue'), color='blue')
    
t = ax.text(
    0.875, 0, "Beam", ha="center", va="center", rotation=0, size=15,
    bbox=dict(boxstyle="rarrow,pad=0.3", fc="cyan", ec="cyan", lw=2))
t2 = ax.text(
    0.125, 0, "Beam", ha="center", va="center", rotation=0, size=15, color='cyan',
    bbox=dict(boxstyle="rarrow,pad=0.3", fc="cyan", ec="cyan", lw=2))
plt.text(0.475, 2.125, r'$B_z$', color='blue', fontsize=14)

xtrace = x[::2000] * 100 + 0.5
ztrace = z[::2000] * 100
plt.plot(xtrace, ztrace, alpha = 0.25)
plt.xlim(0,1.05)
plt.yticks([-1.5,-0.5,0.5,1.5])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.annotate('Electron emitted here', xy=(0.5, 0), xytext=(5, 30), textcoords='offset points', 
                arrowprops=dict(arrowstyle="-|>", color='red'), color='red',
           bbox=dict(boxstyle="round", fc="white", ec="red", lw=0.1))

plt.title('Schematic of Initial simulation', fontsize=16)

plt.tight_layout()
plt.gcf().savefig('beam_sketch.png')

In [ ]:
# Set up coil calculation. Coil in X-Y plane, centred on (0,0)
Ny = 51
Nz = 101
Ntheta = 101
Rcoil = 0.005
I = 40

# Set up a QTNM like field, with coils at +/- 3cm
zc1 = -0.03
zc2 = 0.03
qtnm_bottle = BathTubField(Ntheta, R=Rcoil, I=I, Z1 = zc1, Z2 = zc2, background=B0)

In [ ]:
# Mesh for plotting field
ygrid = np.linspace(-0.05,0.05,Ny)
zgrid = np.linspace(-0.05,0.05,Nz)

Y, Z = np.meshgrid(ygrid, zgrid)

bx = np.zeros_like(Y)
by = np.zeros_like(Y)
bz = np.zeros_like(Y)

# For plotting purposes subtract background back off
for i in range(Nz):
    for j in range(Ny):
        x = 0.0
        y = ygrid[j]
        z = zgrid[i]
        
        bx[i,j], by[i,j], bz[i,j] = qtnm_bottle.evaluate_field_at_point(x, y, z) - B0

In [ ]:
%%time
res1 = solve_3d(3000, b0=B0, v0=v0, mass=me, charge=-qe, tau=tau,
                calc_b_field=qtnm_bottle.evaluate_field_at_point, ic=ic, cfl=1e-1)
print('Final simulated time = %.4Es' % res1.t[-1])

In [ ]:
x = res1.y[0]
y = res1.y[1]
z = res1.y[2]
incr = 10 # Plot every 10th point of trace

plt.streamplot(Y, Z, by, bz, color="blue", linewidth=0.1, density=2)
plt.plot(-Rcoil,zc1, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(Rcoil,zc1, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(-Rcoil,zc2, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(Rcoil,zc2, markersize=3, marker='o', color='orange', alpha=0.75)
plt.axhspan(-0.005, 0.005, color='grey', alpha=0.5)
ax = plt.gca()
t2 = ax.text(
    -0.0325, 0, "Beam", ha="center", va="center", rotation=0, size=15, color='black',
    bbox=dict(boxstyle="rarrow,pad=0.3", fc="cyan", ec="cyan", lw=2,alpha=1.0))

plt.yticks([-0.015,-0.05,0.05,0.015])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

xtrace = x[::incr]
ztrace = z[::incr]
plt.plot(xtrace, ztrace, alpha = 0.25)

plt.xlim(ygrid[0], ygrid[-1])
plt.ylim(zgrid[0], zgrid[-1])
plt.tight_layout()

In [ ]:
# Check what sort of range of vertical field the electron has experienced
plt.plot(zgrid[:], bz[:,25])
plt.axvspan(np.min(res1.y[2]), np.max(res1.y[2]), color='blue', alpha=0.5)

In [ ]:
# Check turning point B-field
t_initial = 0.5 * np.pi - np.deg2rad(1.0)
b_initial = qtnm_bottle.evaluate_field_at_point(0,0,0)[2]
print(b_initial)
b_turning = np.cos(t_initial)**2
print((1.0 + b_turning) * b_initial)
print(qtnm_bottle.evaluate_field_at_point(0,0,np.min(res1.y[2]))[2])

In [ ]:
# All looks promising for trapped electron
# Check vertical motion
plt.plot(res1.t, res1.y[2])
plt.xlim(res1.t[0], res1.t[-1])
plt.xlabel('t(s)', fontsize=14)
plt.ylabel('z(m)', fontsize=14)
plt.title('Vertical motion of trapped electron')
plt.tight_layout()

In [ ]:
%%time

# Shift electron 1cm back
ic = [-0.01, 0, 0, v0 * np.cos(theta), 0, v0 * np.sin(theta), 0.0]
res2 = solve_3d(3000, b0=B0, v0=v0, mass=me, charge=-qe, tau=tau,
                calc_b_field=qtnm_bottle.evaluate_field_at_point, ic=ic, cfl=1e-1)
print('Final simulated time = %.4Es' % res2.t[-1])

In [ ]:
x = res2.y[0]
y = res2.y[1]
z = res2.y[2]

plt.streamplot(Y, Z, by, bz, color="blue", linewidth=0.1, density=2)
plt.plot(-Rcoil,zc1, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(Rcoil,zc1, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(-Rcoil,zc2, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(Rcoil,zc2, markersize=3, marker='o', color='orange', alpha=0.75)
plt.axhspan(-0.005, 0.005, color='grey', alpha=0.5)
ax = plt.gca()
t2 = ax.text(
    -0.0325, 0, "Beam", ha="center", va="center", rotation=0, size=15, color='black',
    bbox=dict(boxstyle="rarrow,pad=0.3", fc="cyan", ec="cyan", lw=2,alpha=1.0))

plt.yticks([-0.015,-0.05,0.05,0.015])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

xtrace = x[::incr]
ztrace = z[::incr]
plt.plot(xtrace, ztrace, alpha = 0.25)

plt.xlim(ygrid[0], ygrid[-1])
plt.ylim(zgrid[0], zgrid[-1])
plt.tight_layout()

In [ ]:
%%time

# Last one not trapped. Try closer
ic = [-0.005, 0, 0, v0 * np.cos(theta), 0, v0 * np.sin(theta), 0.0]
res3 = solve_3d(3000, b0=B0, v0=v0, mass=me, charge=-qe, tau=tau,
                calc_b_field=qtnm_bottle.evaluate_field_at_point, ic=ic, cfl=1e-1)
print('Final simulated time = %.4Es' % res3.t[-1])

In [ ]:
x = res3.y[0]
y = res3.y[1]
z = res3.y[2]

plt.streamplot(Y, Z, by, bz, color="blue", linewidth=0.1, density=2)
plt.plot(-Rcoil,zc1, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(Rcoil,zc1, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(-Rcoil,zc2, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(Rcoil,zc2, markersize=3, marker='o', color='orange', alpha=0.75)
plt.axhspan(-0.005, 0.005, color='grey', alpha=0.5)
ax = plt.gca()
t2 = ax.text(
    -0.0325, 0, "Beam", ha="center", va="center", rotation=0, size=15, color='black',
    bbox=dict(boxstyle="rarrow,pad=0.3", fc="cyan", ec="cyan", lw=2,alpha=1.0))

plt.yticks([-0.015,-0.05,0.05,0.015])
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

xtrace = x[::incr]
ztrace = z[::incr]
plt.plot(xtrace, ztrace, alpha = 0.25)

plt.xlim(ygrid[0], ygrid[-1])
plt.ylim(zgrid[0], zgrid[-1])
plt.tight_layout()

In [ ]:
## Compare our two trapped particles
plt.plot(res3.t, res3.y[2], label='x(t=0) = -0.5cm')
plt.plot(res1.t, res1.y[2], label='x(t=0) = 0')
plt.xlim(res1.t[0], res1.t[-1])
plt.xlabel('t(s)', fontsize=14)
plt.ylabel('z(m)', fontsize=14)
plt.title('Vertical motion of trapped electrons')
plt.legend(loc='upper left')
plt.gcf().set_size_inches(6,4)
plt.tight_layout()

In [ ]:
## Compare our two trapped particles
plt.plot(res3.t, res3.y[5], label='x(t=0) = -0.5cm')
plt.plot(res1.t, res1.y[5], label='x(t=0) = 0')
plt.xlim(res1.t[0], res1.t[-1])
plt.xlabel('t(s)', fontsize=14)
plt.ylabel('vz(m)', fontsize=14)
plt.title('Vertical motion of trapped electrons')
plt.legend(loc='upper left')
plt.gcf().set_size_inches(6,4)
plt.tight_layout()

In [ ]:
# Let's try different starting angles
list_res = []
for theta_deg in np.arange(1,6):
    theta = np.deg2rad(-theta_deg)
    ic = [0, 0, 0, v0 * np.cos(theta), 0, v0 * np.sin(theta), 0.0]
    res_ = solve_3d(3000, b0=B0, v0=v0, mass=me, charge=-qe, tau=tau,
                    calc_b_field=qtnm_bottle.evaluate_field_at_point, ic=ic, cfl=1e-1)
    list_res.append(res_)
    # Add print statement to monitor progress
    print('Final simulated time = %.4Es' % res_.t[-1])

In [ ]:
i = 1
for r in list_res:
    plt.plot(r.t, r.y[2], label='theta = ' + str(i))
    i+=1
    
plt.legend(ncol=2)
plt.xlabel('t(s)', fontsize=14)
plt.ylabel('z(m)', fontsize=14)
plt.ylim(bottom=-0.1)
plt.tight_layout()